In [1]:
import os
import pickle
import numpy as np
import pandas as pd
root = "/home/hemingzhi/.jupyter/ctr"
# table = "xtr_base"
# date = "20210527"
table = 'qp_pv_timeseq'
date = '20210523'
file = os.path.join(root, "data", table, date)

# 1. 看特征的缺失情况
先用 head -n 200000 [file] > test，抽样观察。不要用完整数据

In [23]:
data = pd.read_csv(os.path.join(root, "data", table, "test"), sep='\t')
n = len(data)
cols = []
for fname in data.columns:
    k = len(data[fname][~data[fname].isna()])
#     if k / n >= 0.80:
    print(f"{fname:30} EXIST:{k/n}")
#         cols.append(fname)
# print('\n'.join([f"    , {c}" for c in cols]))
# print('\n'.join([f"    , SUM({c}) AS {c}" for c in cols]))

keyword                        EXIST:1.0
item_id                        EXIST:1.0
show_cnt                       EXIST:1.0
click_cnt                      EXIST:1.0
play_cnt                       EXIST:1.0
like_cnt                       EXIST:1.0
follow_cnt                     EXIST:1.0
long_view_cnt                  EXIST:1.0
short_view_cnt                 EXIST:1.0
first_click_cnt                EXIST:1.0
last_click_cnt                 EXIST:1.0
first_view_long_cnt            EXIST:1.0
last_view_long_cnt             EXIST:1.0
slide_show                     EXIST:1.0
slide_click                    EXIST:1.0
dtype                          EXIST:1.0
now_show_cnt                   EXIST:0.995
now_click_cnt                  EXIST:0.995
now_play_cnt                   EXIST:0.995
now_like_cnt                   EXIST:0.995
now_follow_cnt                 EXIST:0.995
now_long_view_cnt              EXIST:0.995
now_short_view_cnt             EXIST:0.995
now_first_click_cnt            EXIST:0.995


# 2. 看有无重复的键对 (query, item_id)

In [24]:
data['key'] = list(zip(data['keyword'], data['item_id']))
dup = data['key'].duplicated()
print(sum(dup))

875


# 3. 看 query 数，item_id 数

In [25]:
print(len(data))
print(len(data['keyword'].unique()))

2000
330


# 4. 查看样本数

In [4]:
n = 0
file = os.path.join(root, 'data', 'xtr_base', '20210527_filtered_train')
with open(file, 'r', encoding='utf-8') as f:
    for line in f:
        n += 1
print(n)

17193004


# 5. 生成筛选数据，令eval集的 (query, item_id) 全在train集中出现

In [ ]:
vocab = "../../data/vocab/xtr_base_20210527_filtered.pkl"
with open(vocab, 'rb') as pkl:
    sparse_feature_info = pickle.load(pkl)
    # dense_feature_info = pickle.load(pkl)
    # label_feature_info = pickle.load(pkl)
kw_idx = sparse_feature_info['keyword']['index'][0]
it_idx = sparse_feature_info['item_id']['index'][0]

In [3]:
pre_file_root = "../../data/xtr_base/20210527_filtered_train_splits"
pre_files = sorted([os.path.join(pre_file_root, s) for s in os.listdir(pre_file_root)])[:16]

pre_keys = set()
for file in pre_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            feats = line.strip().split('\t')
            pre_keys.add((feats[kw_idx], feats[it_idx]))

In [ ]:
post_file_root = "../../data/xtr_base/20210528_filtered_train_splits"
post_files = sorted([os.path.join(post_file_root, s) for s in os.listdir(post_file_root)])

save_path = "../../data/xtr_base/20210528_keys_in_0527_eval_splits"
os.makedirs(save_path, exist_ok=True)


end = False
source_fp = 0
output_fp = 0
f = open(post_files[source_fp], 'r', encoding='utf-8')
while not end:
    fo = open(os.path.join(save_path, f"20210528_keys_in_0527_{output_fp}"), 'w', encoding='utf-8')
    write_cnt = 0
    while write_cnt < 200000:
        line = f.readline()
        if not line:
            if source_fp + 1 < len(post_files):
                print("DONE:", post_files[source_fp], "NOW_WRITING:", os.path.join(save_path, f"20210528_keys_in_0527_{output_fp}"), "write_cnt:", write_cnt)
                source_fp += 1
                f.close()
                f = open(post_files[source_fp], 'r', encoding='utf-8')
                continue
            else:
                end = True
                break
        try:
            feats = line.strip().split('\t')
            if (feats[kw_idx], feats[it_idx]) in pre_keys:
                fo.write(line)
                write_cnt += 1
        except:
            print(line)
    output_fp += 1
    fo.close()
f.close()

# 6. 统计eval集相比训练集缺失多少(query, item_id)

In [14]:
vocab = "../../data/vocab/xtr_base_20210527_filtered.pkl"
with open(vocab, 'rb') as pkl:
    sparse_feature_info = pickle.load(pkl)
    # dense_feature_info = pickle.load(pkl)
    # label_feature_info = pickle.load(pkl)
kw_idx = sparse_feature_info['keyword']['index'][0]
it_idx = sparse_feature_info['item_id']['index'][0]

pre_file_root = "../../data/xtr_base/20210527_filtered_train_splits"
pre_files = sorted([os.path.join(pre_file_root, s) for s in os.listdir(pre_file_root)])

pre_keys = set()
for file in pre_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            feats = line.strip().split('\t')
            pre_keys.add((feats[kw_idx], feats[it_idx]))

post_file_root = "../../data/xtr_base/20210528_filtered_train_splits"
post_files = sorted([os.path.join(post_file_root, s) for s in os.listdir(post_file_root)])

post_keys = set()
for file in post_files:
    with open(file, 'r', encoding='utf-8') as f:
        for line in f:
            feats = line.strip().split('\t')
            post_keys.add((feats[kw_idx], feats[it_idx]))

In [17]:
print(len(pre_keys))
print(len(post_keys))
print(len(post_keys & pre_keys))
print(len(post_keys & pre_keys) / len(post_keys))

17193003
1212304
834823
0.6886251303303462
